# SWMM experiment procedure

## Steps
0. Calibrate tank and pipe sensors

1. Run an impulse through the system

2. Load impulse into SWMM as inflow hydrograph

3. Compare SWMM pipe depth with measured pipe depth

In [1]:
# Imports.
from urbansurge.swmm_model import SWMM

%load_ext autoreload
%autoreload 2 

## Load in experiment

## Impulse

### Process raw tank measurement to flow.

### Convert to SWMM-readable impulse

### Load SWMM modeled depth

In [ ]:
# Path to configuration file.
config_path = r"lab_system_config.yml"

# Create instance of SWMM model.
swmm = swmm_model.SWMM(config_path)

# Configure model.
swmm.configure_model()

# Run model.
swmm.run_model()

# Extract depth at measurement link.


## Compare measured and modeled depths